<a href="https://colab.research.google.com/github/ssmhrkw/Acoustics_public/blob/master/Revtime_Benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Benchmark problems for Building Acoustics](http://news-sv.aij.or.jp/kankyo/s24/benchmark/a11/a11_j.html)

if you have any question contact me : sh[atmark]kenken.go.jp


参考にしたページ：
[日本音響エンジニアリング株式会社　室内音響の測定　－インパルス応答の読み方－](https://www.noe.co.jp/technology/04/04inv1.html)
[有閑是宝](http://samuiui.com/2019/07/17/インパルス応答から残響時間を計算するpython/)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io.wavfile import read

In [ ]:
# ここのセルは1度以上Runするといっぱい保存される（保存先ってどこなんだろう）
!wget http://news-sv.aij.or.jp/kankyo/s24/benchmark/a11/a11.wav
!wget http://news-sv.aij.or.jp/kankyo/s24/benchmark/a12/a12.wav
!wget http://news-sv.aij.or.jp/kankyo/s24/benchmark/a13/a13.wav

!wget http://news-sv.aij.or.jp/kankyo/s24/benchmark/b11/b11.wav
!wget http://news-sv.aij.or.jp/kankyo/s24/benchmark/b12/b12.wav


!wget http://news-sv.aij.or.jp/kankyo/s24/benchmark/c11/c11.wav
!wget http://news-sv.aij.or.jp/kankyo/s24/benchmark/c12/c12.wav


In [ ]:
def butter_bandpass(lowcut, highcut, order, fs):
    """
    Created on Thu Nov 22
    The function for bandpass filter

    Parameters
    ----------
    lowcut:
        lower frequency side of the butter worth band pass filter in hz
    highcut:
        higher frequency side of the butter worth band pass filter in Hz
    order:
        Order of the butter-worth filter
    fs:
        Sampling frequency
    Returns
    -------
    sos:
        coefficient for sosfilt.
    """

    from scipy.signal import butter
    sos = butter(order, [lowcut / (fs / 2), highcut / (fs / 2)],
                 btype='bandpass',
                 analog=False,
                 output='sos')
    return sos

In [ ]:
def octaveband_filter(data, fs, sec, order, how):
    """
    Created on Thu Nov 22
    Butterworth bandpass filter

    Parameters
    ----------
    data:
        input discrete signal
    fs:
        Sampling frequency
    sec:
        duration of input in integer, (s)
    order:
        ~ order butter-worth filter, usually 6 see - 'https://bksv.com/~/media/literature/Product%20Data/bp0163.ashx'
    how:
        OB (31.5~1000(6) or TOB (25~1600(19))

    Returns
    -------
    fc:
        centre frequency bands (Octave or one-third octave)
    y:
        Filtered signal
    """

    import numpy as np
    from scipy.signal import sosfilt

    if how == 'OB':
        n = 1
        fc = np.array([(10**(i / 10)) * 1000 for i in range(-15, 9, 3)])
        fd = 2**(1 / 2 / n)  # Base 2 (OB)
    elif how == 'TOB':
        n = 3
        fc = np.array([(10**(i / 10)) * 1000 for i in range(-16, 3)])
        fd = 2**(1 / 2 / n)  # Base 10 (TOB)

    flo = fc / fd
    fhi = fc * fd
    lowcut = flo
    highcut = fhi

    y = np.zeros([np.size(fc), int(sec * fs)])
    for i in range(np.size(fc)):
        sos = butter_bandpass(lowcut[i], highcut[i], order, fs)
        y[i, :] = sosfilt(sos, data)
    return y, fc

In [ ]:
def twlf_sos(x, fs, twlf):
    """
    Created on Thu Nov 22
    This function time-weights the CPB-filtered signal

    Parameters
    ----------
    x:
        Discrete signal
    fs:
        Sampling frequency
    twlf:
        time-weighting coefficient 'F' for fast (0.125s) or 'S' for slow (1s)

    Returns
    -------
    y:
        Fast or slow time-weighted discrete signal.(sqrt2 conversion is required)
    """

    from scipy.signal import butter, sosfilt
    import numpy as np

    if twlf == 'F':
        tau = 1 / 8
    elif twlf == 'S':
        tau = 1
    elif twlf == 'RT':
        tau = 12800

    lowcut = 1 / (2 * np.pi * tau)
    sos = butter(1, [lowcut / fs], btype='low', analog=False, output='sos')
    y = sosfilt(sos, x**2)
    y = np.sqrt(y)
    return y

In [ ]:
fs, input_signal = read('c11.wav') #file名はa11,a12,a13,b11,b12,c11,c12のいずれか

query = 'EDT'
sec = (len(input_signal) / fs)

# Octave band pass filter
input_bandpassed, oct_cent_freq = octaveband_filter(
    input_signal, fs, sec, 6, 'OB')  #from 31.5 to 4000k in base 10 exact

if query == 'RT':
    levelstart = -5
    Tx = [5, 10, 20, 30, 60]  # Define Tx
    columns = ['T5', 'T10', 'T20', 'T30', 'T60']

elif query == 'EDT':
    levelstart = 0
    Tx = [10]  # Define Tx
    columns = ['EDT(0to-10dB)']

revtime = np.zeros((len(input_bandpassed), len(Tx)))

for numdldB in range(len(Tx)):
    factor = 60 / Tx[numdldB]
    for j in range(len(input_bandpassed)):

        # compute decay curve
        input_bandpassed_squared = input_bandpassed[j]**2.0
        input_bandpassed_squared_sum = np.sum(input_bandpassed_squared)
        temp = input_bandpassed_squared_sum
        curve = []

        for i in range(len(input_signal)):
            temp = temp - input_bandpassed_squared[i]
            curve.append(temp)

        curve_dB = 10.0 * np.log10(curve)
        normalised_decay_curve = curve_dB - max(curve_dB)

        start = np.argmax(normalised_decay_curve < levelstart, axis=0)
        end = np.argmax(normalised_decay_curve < (-1 * Tx[numdldB] - 5),
                        axis=0)

        Revcurve = normalised_decay_curve[start:end]

        # linear regression for regression target
        x = np.linspace(start, end, end - start)
        a, b = np.polyfit(x, Revcurve, 1)

        # Reverberation time
        revtime[j, numdldB] = (-1.0 * Tx[numdldB] / a) * factor / fs

In [ ]:
outputdf = pd.DataFrame(data=np.round(revtime, 3),
                        index=np.round(oct_cent_freq, 2),
                        columns=columns)
outputdf